In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import seaborn as sns
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import Xception
from sklearn.metrics import accuracy_score

In [ ]:
# # Google colab connection
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/Dataset/data.zip"

In [ ]:
# Enable GPU acceleration
gpus = tf.config.list_logical_devices('GPU')
stg = tf.distribute.MirroredStrategy(gpus)
location = '/content/data'
no_of_classes = 38
batch_size = 32
size = 224

In [4]:
picture_size = (size, size)
train_set = tf.keras.utils.image_dataset_from_directory(
    directory=location,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=22
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory=location,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=22
)

Found 11864 files belonging to 38 classes.
Using 9492 files for training.
Found 11864 files belonging to 38 classes.
Using 2372 files for validation.


In [5]:
# Define the Xception base model
base_model = Xception(weights="imagenet", include_top=False, input_shape=(size, size, 3))

# Create three instances of the base model with different inputs
input_1 = base_model.input
output_1 = base_model.output
input_2 = base_model.input
output_2 = base_model.output
input_3 = base_model.input
output_3 = base_model.output

# Add global average pooling layer and dense layers for each model
output_1 = GlobalAveragePooling2D()(output_1)
output_1 = Dense(512, activation='relu')(output_1)
output_1 = Dropout(0.5)(output_1)
output_1 = Dense(32, activation='relu')(output_1)
output_1 = Dense(no_of_classes, activation='softmax')(output_1)

output_2 = GlobalAveragePooling2D()(output_2)
output_2 = Dense(512, activation='relu')(output_2)
output_2 = Dropout(0.5)(output_2)
output_2 = Dense(32, activation='relu')(output_2)
output_2 = Dense(no_of_classes, activation='softmax')(output_2)

output_3 = GlobalAveragePooling2D()(output_3)
output_3 = Dense(512, activation='relu')(output_3)
output_3 = Dropout(0.5)(output_3)
output_3 = Dense(32, activation='relu')(output_3)
output_3 = Dense(no_of_classes, activation='softmax')(output_3)

# Create three separate models
model_1 = Model(inputs=input_1, outputs=output_1)
model_2 = Model(inputs=input_2, outputs=output_2)
model_3 = Model(inputs=input_3, outputs=output_3)

# Compile the models
model_1.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.001), metrics=['accuracy'])
model_2.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.001), metrics=['accuracy'])
model_3.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.001), metrics=['accuracy'])

In [6]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
callbacks = [lr_scheduler, early_stopping]

In [ ]:
# Train the models
history_1 = model_1.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                        steps_per_epoch=len(train_set), validation_steps=len(validation_set))
history_2 = model_2.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                        steps_per_epoch=len(train_set), validation_steps=len(validation_set))
history_3 = model_3.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                        steps_per_epoch=len(train_set), validation_steps=len(validation_set))

In [ ]:
# Plot Loss and Accuracy per Epoch
plt.figure(figsize=(14, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['val_accuracy'])
plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Model 1 Train', 'Model 1 Val', 'Model 2 Train', 'Model 2 Val', 'Model 3 Train', 'Model 3 Val'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Model 1 Train', 'Model 1 Val', 'Model 2 Train', 'Model 2 Val', 'Model 3 Train', 'Model 3 Val'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Ensemble predictions
ensemble_predictions = []
test_data_dir = location
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Generate predictions for each model and collect them in a list
for model in [model_1, model_2, model_3]:
    predictions = model.predict(test_generator)
    ensemble_predictions.append(predictions)

# Compute the average predictions of the ensemble
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Calculate the accuracy of the ensemble predictions
ensemble_acc = accuracy_score(test_generator.classes, np.argmax(ensemble_predictions, axis=1))
print('Ensemble accuracy:', ensemble_acc)

In [ ]:
# Classification Report
y_true = test_generator.classes
y_pred = np.argmax(ensemble_predictions, axis=1)
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix using Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Calculate overall ROC AUC score
roc_auc = roc_auc_score(tf.keras.utils.to_categorical(y_true), ensemble_predictions, average='macro')
print("Overall ROC AUC Score:", roc_auc)

# Plot ROC curve
fpr, tpr, _ = roc_curve(tf.keras.utils.to_categorical(y_true).ravel(), ensemble_predictions.ravel())
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.5f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()